In [1]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
import plotly.express as px
import plotly.graph_objects as go


import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import  plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_up_to_6
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.backtester.indicators.talib_ind import from_talib
from quantfreedom.backtester.evaluators.evaluators import eval_is_below

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv(
    'E:/Coding/backtesters/QuantFreedom/tests/data/30min.csv', index_col='time')

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=15,
)
rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(40, 61, 10),
)


In [2]:
clear_cache()

In [5]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)
